In [111]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
from pathlib import Path

In [112]:
import os 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
dir_path = os.path.join(dir_path, 'Data')

def get_data(folder_path):
    X = []
    y = []
    dirs = os.listdir(folder_path)
    for path in tqdm(dirs):
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-8") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)
                lines = ' '.join(lines)
                lines = ViTokenizer.tokenize(lines)

                X.append(lines)
                y.append(path)

    return X, y

train_path = os.path.join(dir_path, 'D:\Private\DATA\DATA_LAW\TRAIN_FULL')
X_data, y_data = get_data(train_path)


100%|██████████| 6/6 [00:12<00:00,  2.08s/it]


In [113]:
test_path = os.path.join(dir_path, 'D:\Private\DATA\DATA_LAW\TEST_FULL')
X_test, y_test = get_data(test_path)

100%|██████████| 6/6 [00:05<00:00,  1.18it/s]


In [116]:
def save_to_file(data,file_path):
    with open(file_path,'w',encoding='utf-8') as f:
        for line in data:
            f.write(line + "\n")

save_to_file(X_data,'D://Private//DATA//Model_data//x_train_data.txt')
save_to_file(y_data, 'D://Private//DATA//Model_data//y_train_data.txt')


save_to_file(X_test,'D://Private//DATA//Model_data//x_test_data.txt')
save_to_file(y_test, 'D://Private//DATA//Model_data//y_test_data.txt')

In [117]:
import pickle

pickle.dump(X_data, open('D:/Private/DATA/Model_data/X_data.pkl', 'wb'))
pickle.dump(y_data, open('D:/Private/DATA/Model_data/y_data.pkl', 'wb'))

pickle.dump(X_test, open('D:/Private/DATA/Model_data/X_test.pkl', 'wb'))
pickle.dump(y_test, open('D:/Private/DATA/Model_data/y_test.pkl', 'wb'))

Feature Engineering

In [118]:
import pickle

X_data = pickle.load(open('D:/Private/DATA/Model_data/X_data.pkl', 'rb'))
y_data = pickle.load(open('D:/Private/DATA/Model_data/y_data.pkl', 'rb'))

X_test = pickle.load(open('D:/Private/DATA/Model_data/X_test.pkl', 'rb'))
y_test = pickle.load(open('D:/Private/DATA/Model_data/y_test.pkl', 'rb'))


Count Vectors as features

In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(X_data)
X_test_count = count_vect.transform(X_test)


Tf-Idf Vectors as Features

In [120]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)


In [121]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_data)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)


In [122]:
# ngram-char level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(X_data)
X_data_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_test)


Word Level

In [123]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)


X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)


N-gram Level

In [124]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)

X_data_tfidf_ngram_svd = svd_ngram.transform(X_data_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)


N-gram Char Level

In [125]:
svd_ngram_char = TruncatedSVD(n_components=300, random_state=42)
svd_ngram_char.fit(X_data_tfidf_ngram_char)

X_data_tfidf_ngram_char_svd = svd_ngram_char.transform(X_data_tfidf_ngram_char)
X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)


Word Embeddings

In [126]:

from gensim.models import KeyedVectors 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
word2vec_model_path = os.path.join(dir_path, "DATA/vi.vec")

w2v = KeyedVectors.load_word2vec_format(word2vec_model_path)
vocab = w2v.key_to_index
wv = w2v

def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
                sentence.append(wv[word])

        word2vec_data.append(sentence)

    return word2vec_data

X_data_w2v = get_word2vec_data(X_data)
X_test_w2v = get_word2vec_data(X_test)


In [127]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [128]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))


In [129]:
X_data, y_data, X_test, y_test = X_data_tfidf, y_data, X_test_tfidf, y_test
classifier = LogisticRegression()
model = train_model(classifier,X_data, y_data, X_test, y_test)

Validation accuracy:  0.9797979797979798
Test accuracy:  0.9694117647058823


In [130]:
input_file = 'D:\\Private\\DATA\\bbbb.txt'

with open(input_file,'r',encoding='utf-8') as f:
    text = f.read()
text = ViTokenizer.tokenize(text)

text_to_vect = tfidf_vect.transform([text])
classifier.predict(text_to_vect)

array(['dau_thau'], dtype='<U8')